In [7]:
!pip install --user ipython google-cloud-vision google-cloud

In [1]:
from typing import Sequence

from google.cloud import vision


def analyze_image_from_uri(
    image_uri: str,
    feature_types: Sequence,
) -> vision.AnnotateImageResponse:
    client = vision.ImageAnnotatorClient()

    image = vision.Image()
    image.source.image_uri = image_uri
    features = [vision.Feature(type_=feature_type) for feature_type in feature_types]
    request = vision.AnnotateImageRequest(image=image, features=features)

    response = client.annotate_image(request=request)

    return response


def print_labels(response: vision.AnnotateImageResponse):
    print("=" * 80)
    for label in response.label_annotations:
        print(
            f"{label.score:4.0%}",
            f"{label.description:5}",
            sep=" | ",
        )
        

In [2]:
def detect_text_uri(uri):
    """Detects text in the file located in Google Cloud Storage or on the Web."""
    from google.cloud import vision

    client = vision.ImageAnnotatorClient()
    image = vision.Image()
    image.source.image_uri = uri
    extracted_text=[]

    response = client.text_detection(image=image)
    texts = response.text_annotations
    print("Texts:")

    for text in texts:
        
        extracted_text += {text.description}

    return extracted_text
    if response.error.message:
        raise Exception(
            "{}\nFor more info on error messages, check: "
            "https://cloud.google.com/apis/design/errors".format(response.error.message)
        )

In [3]:
employment_letter = "gs://employment-letter1/Employee-Confirmation-Letter.png"

In [4]:
example = detect_text_uri(employment_letter)

Texts:


In [10]:
employment_letter_content = example[0]

In [6]:
!pip install vertexai google-cloud-aiplatform --upgrade --user

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 10.6 MB/s eta 0:00:00a 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [7]:
PROJECT_ID = "cascades-visionapi"
LOCATION = "us-central1" #e.g. us-central1

import vertexai
vertexai.init(project=PROJECT_ID, location=LOCATION)

In [8]:
import pandas as pd
import seaborn as sns
from IPython.display import Markdown, display
from sklearn.metrics.pairwise import cosine_similarity
from vertexai.preview.language_models import (ChatModel, InputOutputTextPair,
                                              TextEmbeddingModel,
                                              TextGenerationModel)

In [9]:
generation_model = TextGenerationModel.from_pretrained("text-bison@001")

In [14]:
text = f"Full name, Current address, Date submitted, Employer,Employer address, Hiring date,Job title,Employment status, if any of the information is missing, say that is missed, {employment_letter_content}"
prompt = "extract these data points from the text given"
response = generation_model.predict(prompt=f"{prompt}:{text}")

print(response.text)

Full name: Ananya Thakur
Current address: Bangalore
Date submitted: 20th May 2020
Employer: Topper
Employer address: 
Hiring date: 10th January 2020
Job title: Asst. Software Engineer
Employment status: Confirmed


In [16]:
### Letterhead detection

In [17]:
employment_letter2 = "gs://employment-letter1/Confirmation-of-Employment-Letter-Example.jpg"

In [20]:
def detect_logos_uri(uri):
    """Detects logos in the file located in Google Cloud Storage or on the Web."""
    from google.cloud import vision

    client = vision.ImageAnnotatorClient()
    image = vision.Image()
    image.source.image_uri = uri

    response = client.logo_detection(image=image)
    logos = response.logo_annotations
    print("Logos:")

    for logo in logos:
        print(logo.description)

    if response.error.message:
        raise Exception(
            "{}\nFor more info on error messages, check: "
            "https://cloud.google.com/apis/design/errors".format(response.error.message)
        )


In [21]:
detect_logos_uri(employment_letter2)

Logos:
FLSmidth


In [22]:
## Detect signatures / Handwritten 

In [23]:
employment_letter3 = "gs://employment-letter1/employment-letter-signature.png"

In [24]:
def detect_document_uri(uri):
    """Detects document features in the file located in Google Cloud
    Storage."""
    from google.cloud import vision

    client = vision.ImageAnnotatorClient()
    image = vision.Image()
    image.source.image_uri = uri

    response = client.document_text_detection(image=image)

    for page in response.full_text_annotation.pages:
        for block in page.blocks:
            print(f"\nBlock confidence: {block.confidence}\n")

            for paragraph in block.paragraphs:
                print("Paragraph confidence: {}".format(paragraph.confidence))

                for word in paragraph.words:
                    word_text = "".join([symbol.text for symbol in word.symbols])
                    print(
                        "Word text: {} (confidence: {})".format(
                            word_text, word.confidence
                        )
                    )

                    for symbol in word.symbols:
                        print(
                            "\tSymbol: {} (confidence: {})".format(
                                symbol.text, symbol.confidence
                            )
                        )

    if response.error.message:
        raise Exception(
            "{}\nFor more info on error messages, check: "
            "https://cloud.google.com/apis/design/errors".format(response.error.message)
        )



In [25]:
detect_document_uri(employment_letter3)


Block confidence: 0.9823168516159058

Paragraph confidence: 0.9710713028907776
Word text: Tyler (confidence: 0.976246178150177)
	Symbol: T (confidence: 0.9652119278907776)
	Symbol: y (confidence: 0.9575888514518738)
	Symbol: l (confidence: 0.9822619557380676)
	Symbol: e (confidence: 0.9894191026687622)
	Symbol: r (confidence: 0.986748993396759)
Word text: Kennedy (confidence: 0.9673749804496765)
	Symbol: K (confidence: 0.8456246852874756)
	Symbol: e (confidence: 0.9883696436882019)
	Symbol: n (confidence: 0.9927859902381897)
	Symbol: n (confidence: 0.9946004748344421)
	Symbol: e (confidence: 0.993253767490387)
	Symbol: d (confidence: 0.9832133054733276)
	Symbol: y (confidence: 0.9737768173217773)
Paragraph confidence: 0.9860110878944397
Word text: 932 (confidence: 0.9811035990715027)
	Symbol: 9 (confidence: 0.9788810014724731)
	Symbol: 3 (confidence: 0.9801983833312988)
	Symbol: 2 (confidence: 0.9842314124107361)
Word text: Bethesda (confidence: 0.9889155626296997)
	Symbol: B (confide